<a href="https://colab.research.google.com/github/Haeun-Oh/Deep-Learning/blob/main/12_Neural_Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall tensorflow
!pip install tensorflow-gpu==1.13.1

Uninstalling tensorflow-2.5.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.5.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.5.0
     |████████████████████████████████| 345.0MB 47kB/s 
     |████████████████████████████████| 3.2MB 34.9MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 368kB 36.4MB/s 
  Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully unins

In [1]:
import tensorflow as tf
tf.__version__

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

'1.13.1'

In [1]:
import tensorflow as tf
import numpy as np


char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

seq_data = [['word', '단어'], ['wood', '나무'],
            ['game', '놀이'], ['girl', '소녀'],
            ['kiss', '키스'], ['love', '사랑']]

def make_batch(seq_data):
    input_batch =[]
    output_batch = []
    target_batch = []

    for seq in seq_data:
        input = [num_dic[n] for n in seq[0]]
        output = [num_dic[n] for n in ('S' + seq[1])]
        target = [num_dic[n] for n in (seq[1]+'E')]

        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])

        target_batch.append(target)
    return input_batch, output_batch, target_batch

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [2]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 100

n_class = n_input = dic_len

enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32,[None, None, n_input])

targets = tf.placeholder(tf.int64,[None, None])

with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)

    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype=tf.float32)

with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)

    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, initial_state=enc_states, dtype=tf.float32)

model = tf.layers.dense(outputs, n_class, activation=None)
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=targets))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.


In [3]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict={enc_input: input_batch, dec_input: output_batch, targets: target_batch})

    print('Epoch:', '%04d' %(epoch + 1), 'cost =', '{:.6f}'.format(loss))

print('최적화 완료!')

def translate(word):
    seq_data = [word, 'p' * len(word)]

    input_batch, output_batch, target_batch = make_batch([seq_data])
    prediction = tf.argmax(model, 2)

    result = sess.run(prediction, feed_dict={enc_input: input_batch, dec_input: output_batch, targets: target_batch})

    decoded = [char_arr[i] for i in result[0]]

    end = decoded.index('E')
    translated = ''.join(decoded[:end])

    return translated

Epoch: 0001 cost = 3.824376
Epoch: 0002 cost = 2.643504
Epoch: 0003 cost = 1.655210
Epoch: 0004 cost = 1.395490
Epoch: 0005 cost = 0.772846
Epoch: 0006 cost = 0.441861
Epoch: 0007 cost = 0.512418
Epoch: 0008 cost = 0.204965
Epoch: 0009 cost = 0.274956
Epoch: 0010 cost = 0.153133
Epoch: 0011 cost = 0.184910
Epoch: 0012 cost = 0.081037
Epoch: 0013 cost = 0.129175
Epoch: 0014 cost = 0.077806
Epoch: 0015 cost = 0.039698
Epoch: 0016 cost = 0.039521
Epoch: 0017 cost = 0.049575
Epoch: 0018 cost = 0.086784
Epoch: 0019 cost = 0.019368
Epoch: 0020 cost = 0.022847
Epoch: 0021 cost = 0.006999
Epoch: 0022 cost = 0.008751
Epoch: 0023 cost = 0.021318
Epoch: 0024 cost = 0.009220
Epoch: 0025 cost = 0.008988
Epoch: 0026 cost = 0.015099
Epoch: 0027 cost = 0.009495
Epoch: 0028 cost = 0.001419
Epoch: 0029 cost = 0.005977
Epoch: 0030 cost = 0.002236
Epoch: 0031 cost = 0.002581
Epoch: 0032 cost = 0.004072
Epoch: 0033 cost = 0.002623
Epoch: 0034 cost = 0.001790
Epoch: 0035 cost = 0.012290
Epoch: 0036 cost = 0

In [4]:
print('\n=== 번역 테스트 ===')

print('word -> ', translate('word'))
print('wodr -> ', translate('wodr'))
print('love -> ', translate('love'))
print('loev -> ', translate('loev'))
print('abcd -> ', translate('abcd'))



=== 번역 테스트 ===
word ->  단어
wodr ->  나무
love ->  사랑
loev ->  사랑
abcd ->  사랑랑
